In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pro_data

+ 第一个修改特征
# 项目数据6project.csv提取的特征
    1. 项目的数量个数， PROJECT_CNT,
    2. 省内项目的个数， PROJECT_INHOME_CNT
    3. 省外项目的个数， PROJECT_OUTHOME_CNT
    4. 省内项目的比例， PROJECT_INHOME_RATE
    5. 省外项目的比例， PROJECT_OUTHOME_RATE
    
    6. 项目的类型数目，PROJECT_TYPECODE_CNT

    7. 第一个项目的项目ID，PROJECT_FIRST_TYPECODE
    8. 最后一个项目的项目ID，PROJECT_END_TYPECODE
    
    9. 第一个项目中标的时间，PROJECT_FIRST_TIME
    10. 最后一个项目中标的时间，PROJECT_END_TIME
    
    11. 最后一个项目中标时间和第一个中标时间的差值，PROJECT_END_FIRST_DIFF
    12. 第一个项目中标时间距离企业注册时间的差值，PROJECT_FIRST_RGYEAR_DIFF
    13. 最后一个项目中标时间距离企业注册时间的差值，PROJECT_END_RGYEAR_DIFF


    14. 第一个项目中标时间距离企业第一个变更时间的差值，PROJECT_FIRST_CHANGE_FIRST_DIFF
    15. 最后一个项目中标时间距离企第一个变更时间的差值，PROJECT_END_CHANGE_FIRST_DIFF
    16. 第一个项目中标时间距离企业最后一个变更时间的差值，PROJECT_FIRST_CHANGE_END_DIFF
    17. 最后一个项目中标时间距离企最后一个变更时间的差值，PROJECT_END_CHANGE_END_DIFF
    
    ---------------------------------------------------------------------------------
    18. 平均没几个月中标一个项目其余用0表示，PROJECT_PRE_MONTH_CNT = PROJECT_END_RGYEAR_DIFF / PROJECT_CNT

    19. 企业权利的个数占所有权利个数平均值的比例。PROJECT_CNT_ALL_RATE
    
    20. 企业对应的大类HY的平均权利的个数，PROJECT_HY_CNT_AVG
    21. 企业对应大类HY的平均权利个数占所有权利平均个数的比例，PROJECT_HY_CNT_ALL_RATE
    22. 企业权利的个数占其对应的大类HY的平均值的比例，PROJECT_CNT_HY_RATE

    
    

In [2]:
df_all = pd.read_csv("../data/alldata/df_data12345.csv")
df_project = pd.read_csv("../data/public/6project.csv")

In [3]:
df_all.info()
df_all.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255130 entries, 0 to 255129
Columns: 138 entries, EID to RIGHT_CNT_HY_RATE
dtypes: float64(88), int64(37), object(13)
memory usage: 268.6+ MB


,EID,RGYEAR,HY,ZCZB,ETYPE,MPNUM,INUM,FINZB,FSTINUM,TZINUM,...,RIGHT_END_ASK_FIRST_CHANGE_DIFF,RIGHT_END_FB_FIRST_CHANGE_DIFF,RIGHT_FIRST_ASK_END_CHANGE_DIFF,RIGHT_FIRST_FB_END_CHANGE_DIFF,RIGHT_END_ASK_END_CHANGE_DIFF,RIGHT_END_FB_END_CHANGE_DIFF,RIGHT_CNT_ALL_RATE,RIGHT_HY_CNT_AVG,RIGHT_HY_CNT_ALL_RATE,RIGHT_CNT_HY_RATE
0,32741,2000-01,87,100.0,7,1.0,3.0,0.0,2.0,0.0,...,5,5,-6,-6,5,5,0.571313,7.014960,0.571313,1.000000
1,294220,2003-01,51,50.0,7,0.0,3.0,0.0,0.0,0.0,...,-124,-94,-124,-104,-124,-94,0.162884,4.423446,0.360254,0.452136
2,10526,2013-01,75,100.0,7,1.0,2.0,0.0,1.0,0.0,...,5,5,-6,-10,5,5,2.533139,31.103584,2.533139,1.000000
3,237382,2014-01,75,9900.0,7,3.0,4.0,0.0,2.0,0.0,...,2,4,2,16,2,4,0.244326,31.103584,2.533139,0.096452
4,473535,2008-01,75,50.0,7,3.0,5.0,0.0,1.0,0.0,...,5,5,-6,-10,5,5,2.533139,31.103584,2.533139,1.000000


In [4]:
df_project.info()
df_project.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32826 entries, 0 to 32825
Data columns (total 4 columns):
EID         32826 non-null int64
TYPECODE    32826 non-null int64
DJDATE      32826 non-null object
IFHOME      32826 non-null int64
dtypes: int64(3), object(1)
memory usage: 1.0+ MB


,EID,TYPECODE,DJDATE,IFHOME
0,3273,5523256,2015-07,0
1,3649,4073327,2014-03,1
2,4253,4445461,2014-08,0
3,4595,3184459,2013-12,0
4,4595,3214578,2013-12,0


In [5]:
len(df_project['TYPECODE'].value_counts())

28575

In [7]:
EIDS = set(df_project['EID'])

print(len(EIDS))

columns = df_project.columns
df_xproject = pd.DataFrame(columns=columns)

print(columns)

5199
Index(['EID', 'TYPECODE', 'DJDATE', 'IFHOME'], dtype='object')


In [8]:
k = 0
for EID in EIDS:
    if k%3000 == 0:
        print('第%d次处理--------->',k)
    k+=1
    tmp = df_project[df_project['EID'] == EID]
    row = [EID,tmp['TYPECODE'].values,tmp['DJDATE'].values,tmp['IFHOME'].values]
    
    df_xproject = df_xproject.append(pd.Series(row,columns),ignore_index=True)
    

第%d次处理---------> 0
第%d次处理---------> 3000


In [9]:
df_xproject.info()
df_xproject.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5199 entries, 0 to 5198
Data columns (total 4 columns):
EID         5199 non-null float64
TYPECODE    5199 non-null object
DJDATE      5199 non-null object
IFHOME      5199 non-null object
dtypes: float64(1), object(3)
memory usage: 162.5+ KB


,EID,TYPECODE,DJDATE,IFHOME
0,442371.0,"[4608116, 4528561, 4249574, 4085292, 5510188, ...","[2014-10, 2014-09, 2014-06, 2014-04, 2015-07, ...","[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1]"
1,483333.0,[5220850],[2015-04],[1]
2,5.0,"[3043993, 3179032, 3021080, 3008091, 3244329, ...","[2013-10, 2013-12, 2013-09, 2013-09, 2013-12, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,417798.0,"[4995335, 4899767, 4962907, 4893656]","[2015-02, 2015-01, 2015-01, 2014-12]","[1, 1, 1, 1]"
4,270344.0,"[3215071, 3180516, 3201216, 3100025, 4861189, ...","[2013-12, 2013-12, 2013-12, 2013-11, 2014-12, ...","[0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, ..."


In [12]:
df_xproject['PROJECT_CNT'] = df_xproject['TYPECODE'].apply(lambda x: len(x))
df_xproject['PROJECT_INHOME_CNT'] = df_xproject['IFHOME'].apply(lambda x: list(x).count(1))
df_xproject['PROJECT_OUTHOME_CNT'] = df_xproject['IFHOME'].apply(lambda x: list(x).count(0))


In [13]:
df_xproject['PROJECT_INHOME_RATE'] = df_xproject['PROJECT_INHOME_CNT'] / df_xproject['PROJECT_CNT']
df_xproject['PROJECT_OUTHOME_RATE'] = df_xproject['PROJECT_OUTHOME_CNT'] / df_xproject['PROJECT_CNT']



In [14]:
df_xproject['PROJECT_TYPECODE_CNT'] = df_xproject['TYPECODE'].apply(lambda x: len(set(x)) )

df_xproject['PROJECT_FIRST_TYPECODE'] = df_xproject['TYPECODE'].apply(lambda x: x[0])
df_xproject['PROJECT_END_TYPECODE'] = df_xproject['TYPECODE'].apply(lambda x: x[-1])

df_xproject['PROJECT_FIRST_TIME'] = df_xproject['DJDATE'].apply(lambda x: x[0])
df_xproject['PROJECT_END_TIME'] = df_xproject['DJDATE'].apply(lambda x: x[-1])


In [58]:
df_xproject.to_csv('../data/public/6project_1.csv',index=False,index_label=False)
df_xproject.columns



Index(['EID', 'TYPECODE', 'DJDATE', 'IFHOME', 'PROJECT_CNT',
       'PROJECT_INHOME_CNT', 'PROJECT_OUTHOME_CNT', 'PROJECT_INHOME_RATE',
       'PROJECT_OUTHOME_RATE', 'PROJECT_TYPECODE_CNT',
       'PROJECT_FIRST_TYPECODE', 'PROJECT_END_TYPECODE', 'PROJECT_FIRST_TIME',
       'PROJECT_END_TIME'],
      dtype='object')

In [15]:
df_all = pd.merge(df_all,df_xproject[['EID', 'PROJECT_CNT','PROJECT_INHOME_CNT', 'PROJECT_OUTHOME_CNT', 'PROJECT_INHOME_RATE',
                                    'PROJECT_OUTHOME_RATE', 'PROJECT_TYPECODE_CNT', 'PROJECT_FIRST_TYPECODE', 'PROJECT_END_TYPECODE',
                                    'PROJECT_FIRST_TIME','PROJECT_END_TIME']],how='left',on=['EID'])


In [16]:
df_all.info()
df_all.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255130 entries, 0 to 255129
Columns: 148 entries, EID to PROJECT_END_TIME
dtypes: float64(96), int64(37), object(15)
memory usage: 290.0+ MB


,EID,RGYEAR,HY,ZCZB,ETYPE,MPNUM,INUM,FINZB,FSTINUM,TZINUM,...,PROJECT_CNT,PROJECT_INHOME_CNT,PROJECT_OUTHOME_CNT,PROJECT_INHOME_RATE,PROJECT_OUTHOME_RATE,PROJECT_TYPECODE_CNT,PROJECT_FIRST_TYPECODE,PROJECT_END_TYPECODE,PROJECT_FIRST_TIME,PROJECT_END_TIME
0,32741,2000-01,87,100.0,7,1.0,3.0,0.0,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,294220,2003-01,51,50.0,7,0.0,3.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10526,2013-01,75,100.0,7,1.0,2.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,237382,2014-01,75,9900.0,7,3.0,4.0,0.0,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,473535,2008-01,75,50.0,7,3.0,5.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# 空值填充，根据HY的类别的平均值或者众数进行填充
# ['RIGHT_CNT','RIGHT_TYPE_CNT', 'RIGHT_TYPE_RATE']使用同一个大类别的均值进行填充
# ['RIGHT_FIRST_TYPECODE','RIGHT_END_TYPECODE', 'RIGHT_TYPECODE_MUCHID', 'RIGHT_FIRST_ASK_TIME',
#     'RIGHT_FIRST_FB_TIME', 'RIGHT_END_ASK_TIME', 'RIGHT_END_FB_TIME']使用同一个大类别的众数进行填充

HYLIST = set(df_all['HY'])
print(HYLIST)

meanlist = ['PROJECT_FIRST_TYPECODE', 'PROJECT_END_TYPECODE','PROJECT_CNT','PROJECT_INHOME_CNT', 'PROJECT_OUTHOME_CNT','PROJECT_OUTHOME_CNT','PROJECT_OUTHOME_RATE','PROJECT_TYPECODE_CNT']
modelist = ['PROJECT_FIRST_TIME', 'PROJECT_END_TIME']


{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 93}


In [18]:
# 所有的关于时间的空值用2018-01填充，假设这些企业被执行的数据发生在下一年，其他用0填充便是没有被执行

for d in meanlist:
    df_all.loc[df_all[d].isnull(),d] = 0

for c in modelist:
    df_all.loc[df_all[c].isnull(),c] = '2018-01'


In [19]:
def timeDiff(x):
    a = x[:x.find(':')]
    b = x[x.find(':')+1:]
    y = int(a[:a.find('-')]) - int(b[:b.find('-')])
    m = int(a[a.find('-')+1:]) - int(b[b.find('-')+1:])
    return y * 12 + m


In [20]:
df_all['PROJECT_END_FIRST_DIFF'] = (df_all['PROJECT_END_TIME'] + ':' + df_all['PROJECT_FIRST_TIME']).apply(timeDiff)

df_all['PROJECT_FIRST_RGYEAR_DIFF'] = (df_all['PROJECT_FIRST_TIME'] + ':' + df_all['RGYEAR']).apply(timeDiff)
df_all['PROJECT_END_RGYEAR_DIFF'] = (df_all['PROJECT_END_TIME'] + ':' + df_all['RGYEAR']).apply(timeDiff)

df_all['PROJECT_FIRST_CHANGE_FIRST_DIFF'] = (df_all['PROJECT_FIRST_TIME'] + ':' + df_all['FIRST_CHANGE_TIME']).apply(timeDiff)
df_all['PROJECT_END_CHANGE_FIRST_DIFF'] = (df_all['PROJECT_END_TIME'] + ':' + df_all['FIRST_CHANGE_TIME']).apply(timeDiff)

df_all['PROJECT_FIRST_CHANGE_END_DIFF'] = (df_all['PROJECT_FIRST_TIME'] + ':' + df_all['END_CHANGE_TIME']).apply(timeDiff)
df_all['PROJECT_END_CHANGE_END_DIFF'] = (df_all['PROJECT_END_TIME'] + ':' + df_all['END_CHANGE_TIME']).apply(timeDiff)


In [22]:
df_all['PROJECT_PRE_MONTH_CNT'] = df_all['PROJECT_END_RGYEAR_DIFF'] / df_all['PROJECT_CNT']
df_all['PROJECT_CNT_ALL_RATE'] = df_all['PROJECT_CNT'] / df_all['PROJECT_CNT'].mean()


In [23]:
tmp = pd.DataFrame()

tmp['PROJECT_HY_CNT_AVG'] = df_all.groupby(['HY'],as_index=True,axis=0)['PROJECT_CNT'].mean()
tmp['PROJECT_HY_CNT_ALL_RATE'] = tmp['PROJECT_HY_CNT_AVG'] / df_all['PROJECT_CNT'].mean()

tmp['HY'] = tmp.index

df_all = pd.merge(df_all,tmp,how='left',on=['HY'])



In [24]:
df_all['PROJECT_CNT_HY_RATE'] = df_all['PROJECT_CNT'] / df_all['PROJECT_HY_CNT_AVG']


In [25]:
df_all.info()
df_all.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 255130 entries, 0 to 255129
Columns: 160 entries, EID to PROJECT_CNT_HY_RATE
dtypes: float64(101), int64(44), object(15)
memory usage: 313.4+ MB


,EID,RGYEAR,HY,ZCZB,ETYPE,MPNUM,INUM,FINZB,FSTINUM,TZINUM,...,PROJECT_END_RGYEAR_DIFF,PROJECT_FIRST_CHANGE_FIRST_DIFF,PROJECT_END_CHANGE_FIRST_DIFF,PROJECT_FIRST_CHANGE_END_DIFF,PROJECT_END_CHANGE_END_DIFF,PROJECT_PRE_MONTH_CNT,PROJECT_CNT_ALL_RATE,PROJECT_HY_CNT_AVG,PROJECT_HY_CNT_ALL_RATE,PROJECT_CNT_HY_RATE
0,32741,2000-01,87,100.0,7,1.0,3.0,0.0,2.0,0.0,...,216,34,34,34,34,inf,0.0,0.030336,0.235777,0.0
1,294220,2003-01,51,50.0,7,0.0,3.0,0.0,0.0,0.0,...,180,34,34,34,34,inf,0.0,0.123225,0.957728,0.0
2,10526,2013-01,75,100.0,7,1.0,2.0,0.0,1.0,0.0,...,60,34,34,34,34,inf,0.0,0.254631,1.979042,0.0
3,237382,2014-01,75,9900.0,7,3.0,4.0,0.0,2.0,0.0,...,48,34,34,34,34,inf,0.0,0.254631,1.979042,0.0
4,473535,2008-01,75,50.0,7,3.0,5.0,0.0,1.0,0.0,...,120,34,34,34,34,inf,0.0,0.254631,1.979042,0.0


In [26]:
# 得到在df_xchange的所有数据，
df_all = df_all.fillna(0)
df_all.to_csv('../data/alldata/df_data123456.csv',index=False,index_label=False)



<class 'pandas.core.frame.DataFrame'>
Int64Index: 255130 entries, 0 to 255129
Columns: 155 entries, EID to PROJECT_END_CHANGE_END_DIFF
dtypes: float64(96), int64(44), object(15)
memory usage: 303.7+ MB


,EID,RGYEAR,HY,ZCZB,ETYPE,MPNUM,INUM,FINZB,FSTINUM,TZINUM,...,PROJECT_END_TYPECODE,PROJECT_FIRST_TIME,PROJECT_END_TIME,PROJECT_END_FIRST_DIFF,PROJECT_FIRST_RGYEAR_DIFF,PROJECT_END_RGYEAR_DIFF,PROJECT_FIRST_CHANGE_FIRST_DIFF,PROJECT_END_CHANGE_FIRST_DIFF,PROJECT_FIRST_CHANGE_END_DIFF,PROJECT_END_CHANGE_END_DIFF
0,32741,2000-01,87,100.0,7,1.0,3.0,0.0,2.0,0.0,...,0.0,2018-01,2018-01,0,216,216,34,34,34,34
1,294220,2003-01,51,50.0,7,0.0,3.0,0.0,0.0,0.0,...,0.0,2018-01,2018-01,0,180,180,34,34,34,34
2,10526,2013-01,75,100.0,7,1.0,2.0,0.0,1.0,0.0,...,0.0,2018-01,2018-01,0,60,60,34,34,34,34
3,237382,2014-01,75,9900.0,7,3.0,4.0,0.0,2.0,0.0,...,0.0,2018-01,2018-01,0,48,48,34,34,34,34
4,473535,2008-01,75,50.0,7,3.0,5.0,0.0,1.0,0.0,...,0.0,2018-01,2018-01,0,120,120,34,34,34,34
